In [1]:
client_id = ''
client_secret = ''

import pandas as pd
from datetime import datetime, timedelta
import requests
import re
def handle_authorization(data):
    
    access_token = data["access_token"]
    print(f" {access_token}")
    return access_token
    
def get_twitch_authorization(client_id, client_secret):
    url = f"https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
    }
    
    response = requests.post(url, params=params)
    data = response.json()
    
    handle_authorization(data)



# Obtén la autorización de Twitch
get_twitch_authorization(client_id, client_secret)

 9ecr4n5dmmd91nxxpy5pn3n5rm7dfc


In [2]:
oauth_token = ''

In [3]:
def extract_channel_name(url):
    pattern = r"https?:\/\/(?:www\.|go\.)?twitch\.tv\/([a-zA-Z0-9_]{4,25})"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    return None
def get_streamer_data(client_id, url):
    channel_name = extract_channel_name(url)
    if not channel_name:
        return None
    
    url = f"https://api.twitch.tv/helix/users"
    
    params = {
        "login": channel_name
    }
    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }
    
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    print(data)
    if "data" in data and len(data["data"]) > 0:
        user_id = data["data"][0]["id"]
        stream_url = f"https://api.twitch.tv/helix/streams?user_id={user_id}"
        response = requests.get(stream_url, headers=headers)
        stream_data = response.json()
        
        if "data" in stream_data and len(stream_data["data"]) > 0:
            return stream_data["data"][0]
    
    return None
url = "https://www.twitch.tv/Perxitaa"
streamer_data = get_streamer_data(client_id, url)

if streamer_data:
    data = f"Datos del streamer {channel_url}:"

{'data': [{'id': '35980866', 'login': 'perxitaa', 'display_name': 'Perxitaa', 'type': '', 'broadcaster_type': 'partner', 'description': 'Troll de Youtube http://www.youtube.com/user/perxitaa', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/6213a284-c133-41cf-ad99-ebef738eb550-profile_image-300x300.png', 'offline_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/c9422bb0-7ffe-442c-9c3b-63633cef47d8-channel_offline_image-1920x1080.jpeg', 'view_count': 0, 'created_at': '2012-09-05T22:12:47Z'}]}


In [4]:
data = {'data': [{'id': '35980866', 'login': 'perxitaa', 'display_name': 'Perxitaa', 'type': '', 'broadcaster_type': 'partner', 'description': 'Troll de Youtube http://www.youtube.com/user/perxitaa', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/6213a284-c133-41cf-ad99-ebef738eb550-profile_image-300x300.png', 'offline_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/c9422bb0-7ffe-442c-9c3b-63633cef47d8-channel_offline_image-1920x1080.jpeg', 'view_count': 0, 'created_at': '2012-09-05T22:12:47Z'}]}
data_dict = data['data'][0]
id_canal =  list(data_dict.values())[0]
id_canal

'35980866'

In [5]:
def obtener_vods(id_canal):
    # Calcular la fecha hace un año desde hoy
    fecha_inicio = datetime.now() - timedelta(days=730)
    fecha_inicio_str = fecha_inicio.strftime("%Y-%m-%dT%H:%M:%SZ")

    url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100"

    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }

    vods_data = []

    while url:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            vods = data["data"]

            for vod in vods:
                vod_id = int(vod["id"])
                fecha_inicio = datetime.strptime(vod["created_at"], "%Y-%m-%dT%H:%M:%SZ")
                fecha_final = datetime.strptime(vod["published_at"], "%Y-%m-%dT%H:%M:%SZ")
                duracion = vod["duration"]
                visitas = int(vod["view_count"])
                nombre = vod["title"].lower().replace(' ', '_')
                descripcion = vod["description"]
                url_vod = f"https://www.twitch.tv/videos/{vod_id}"

                vods_data.append({
                    "ID del VOD": vod_id,
                    "URL del VOD": url_vod,
                    "Nombre del VOD": nombre,
                    "Duración": duracion,
                    "Fecha de inicio": fecha_inicio,
                    "Fecha de finalización": fecha_final,
                    "Visitas": visitas,
                    
                })

            pagination = data.get("pagination", None)
            if pagination and "cursor" in pagination:
                cursor = pagination["cursor"]
                url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100&after={cursor}"
            else:
                url = None

        else:
            print("Se produjo un error al realizar la solicitud GET.")
            return None

    # Crear un DataFrame a partir de los datos
    df_vods = pd.DataFrame(vods_data)

    return df_vods

df = obtener_vods(id_canal)
df.info(memory_usage='deep')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID del VOD             44 non-null     int64         
 1   URL del VOD            44 non-null     object        
 2   Nombre del VOD         44 non-null     object        
 3   Duración               44 non-null     object        
 4   Fecha de inicio        44 non-null     datetime64[ns]
 5   Fecha de finalización  44 non-null     datetime64[ns]
 6   Visitas                44 non-null     int64         
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 13.0 KB


,ID del VOD,URL del VOD,Nombre del VOD,Duración,Fecha de inicio,Fecha de finalización,Visitas
0,1890110941,https://www.twitch.tv/videos/1890110941,el_ultimo_dia_antes_de_vacaciones!!!,4h20m14s,2023-08-04 18:54:56,2023-08-04 18:54:56,63499
1,1888281970,https://www.twitch.tv/videos/1888281970,como_fueron_fichajes_kings_league_troncos_fc!!!,3h1m26s,2023-08-02 16:57:50,2023-08-02 16:57:50,54357
2,1886595517,https://www.twitch.tv/videos/1886595517,descartes_fichaje_troncos_y_luego_volkacio_esp...,6h45m12s,2023-07-31 17:32:54,2023-07-31 17:32:54,159546
3,1885645641,https://www.twitch.tv/videos/1885645641,descartes_futuro_troncos_y_troncas_-_final_kin...,2h26m7s,2023-07-30 14:38:50,2023-07-30 14:38:50,67391
4,1884962370,https://www.twitch.tv/videos/1884962370,final_kings_y_queens_league_vamos_troncas_fc!!!,3h32m8s,2023-07-29 18:44:15,2023-07-29 18:44:15,91413


In [6]:
df.to_csv('twitch_api_kingsleague_csv.csv', index=False)